<h2 style="font-weight: bold">Store Sales TS Forecast</h2>

<h4>Emmm... wouldn't it be if we could use machine learning to predict grocery sales. So Yeah! you guess it! this notebook is going to be about the Store Sales TS Forecast Compeition 😄😄<br><br>I will be doing a EDA of review texts, some Visualization and Pre-Processing. and finally modelling <br></h4>

* <h5 style="font-weight: 700">Your feedback is very welcome</h5>
* <h5 style="font-weight: 700">If you find this notebook useful, please don't forget to upvote it!</h5>

In [ ]:
# Required packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Loading the Data
oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")
holidays = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")
samp_subm = pd.read_csv("../input/store-sales-time-series-forecasting/sample_submission.csv")

#### **Getting to know the Data**

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
oil.head()

In [ ]:
holidays.head()

In [ ]:
stores.head()

In [ ]:
transactions.head()

#  **Exploratory Data Analysis**

In [ ]:
# This function will do basic checks for Dataframes
def basic_eda(df):
    print("\n----------TOP 5 RECORDS--------\n")
    print(df.head(5))
    print("\n----------INFO-----------------\n")
    print(df.info())
    print("\n----------Describe-------------\n")
    print(df.describe())
    print("\n----------Columns--------------\n")
    print(df.columns)
    print("\n----------Data Types-----------\n")
    print(df.dtypes)
    print("\n-------Missing Values----------\n")
    print(df.isnull().sum())
    print("\n-------NULL values-------------\n")
    print(df.isna().sum())
    print("\n-----Shape Of Data-------------\n")
    print(df.shape)
    numeric_variables = df.select_dtypes(include=[np.number])
    categorical_variables = df.select_dtypes(exclude=[np.number])

    print('Numeric variables in the given the dataframe : ',numeric_variables.shape[1])
    print('Categorical variables in the givne the dataframe:',categorical_variables.shape[1])

In [ ]:
#Litle bit of exploration of data

print("\n============================= oil =============================")
basic_eda(oil)

In [ ]:
print("\n============================= holidays =============================")
basic_eda(holidays)

In [ ]:
print("\n============================= Shops =============================")
basic_eda(stores)

In [ ]:
print("\n============================= train =============================")
basic_eda(train)

In [ ]:
print("\n============================= test =============================")
basic_eda(test)

In [ ]:
print("\n============================= transactions =============================")
basic_eda(transactions )

In [ ]:
# Let's check the store data
stores.type.value_counts()

In [ ]:
stores.state.value_counts()

In [ ]:
stores.city.value_counts()

#### **Let's do some visualization**

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(data=stores, x='type', order=stores.type.value_counts().index)

plt.title('Number of Stores based on Type',fontweight="bold")
plt.xlabel('Type')
plt.ylabel('Count')

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(data=stores, y='city',order=stores.city.value_counts().index)

plt.title('Number of Stores based on Cities',fontweight="bold")
plt.ylabel('City')
plt.xlabel('Count')

In [ ]:
train.date = pd.to_datetime(train.date)
test.date = pd.to_datetime(test.date)
months_sales = train.groupby(train['date'].dt.strftime('%B'))['sales'].sum().sort_values()

plt.figure(figsize=(15,10))
plt.plot(months_sales)
plt.title("the sales by month", fontweight="bold")
plt.xlabel("the month")
plt.ylabel("the sales")

In [ ]:
plt.figure(figsize=(15,8))
train.groupby(pd.Grouper(key='date', freq='2D')).sum()['sales'].plot()
plt.ylabel("the sales", fontsize=15)

In [ ]:
plt.figure(figsize=(15,10))
train.groupby(pd.Grouper(key='date', freq='1Y')).sum()['sales'].plot()
plt.title("the sales by year", fontweight="bold")
plt.ylabel("the sales")

#  **Pre-processing & Feature Engineering**

In [ ]:
# some Features
features = ['store_nbr', 'family', 'onpromotion']
target = 'sales'

In [ ]:
# creating the features weekday, month or year.
def extract_weekday(df):
    return df.dayofweek

def extract_month(df):
    return df.month

def extract_year(df):
    return df.year

In [ ]:
train['weekday'] = train['date'].apply(extract_weekday)
train['year'] = train['date'].apply(extract_year)
train['month'] = train['date'].apply(extract_month)

test['weekday'] = test['date'].apply(extract_weekday)
test['year'] = test['date'].apply(extract_year)
test['month'] = test['date'].apply(extract_month)

In [ ]:
features.append('weekday')
features.append('year')
features.append('month')

#### **Encode Labels**

In [ ]:
store_sales = pd.read_csv(
    '../input/store-sales-time-series-forecasting/train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()


df_test = pd.read_csv(
    '../input/store-sales-time-series-forecasting/test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [ ]:
# Target
y = store_sales.unstack(['store_nbr', 'family']).loc["2017"]
y

#### **Create training data**

In [ ]:
fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()
X['NewYear'] = (X.index.dayofyear == 1)

#  **Modeling**

In [ ]:
# Linear Regression
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [ ]:
X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'
X_test['NewYear'] = (X_test.index.dayofyear == 1)

#### **Submission**

In [ ]:
y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)

#### **Credit**
https://www.kaggle.com/dkomyagin/simple-ts-ridge-rf